In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
from sklearn import datasets
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline


In [23]:
import geopandas as gpd

In [5]:
df1 = gpd.read_file("NYC Taxi Zones.geojson")

df1.head(2)

,shape_area,objectid,shape_leng,location_id,zone,borough,geometry
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ..."
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."


In [6]:
import geopandas as gpd

In [7]:
df = gpd.read_file("NYC Taxi Zones.geojson")

df.head(2)

df["lon"] = df["geometry"].centroid.x
df["lat"] = df["geometry"].centroid.y
df

<ipython-input-7-da262ff1d4a7>:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df["lon"] = df["geometry"].centroid.x
<ipython-input-7-da262ff1d4a7>:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df["lat"] = df["geometry"].centroid.y


,shape_area,objectid,shape_leng,location_id,zone,borough,geometry,lon,lat
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ...",-74.174000,40.691831
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ...",-73.831299,40.616745
2,0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ...",-73.847422,40.864474
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ...",-73.976968,40.723752
4,0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island,"MULTIPOLYGON (((-74.17422 40.56257, -74.17349 ...",-74.188484,40.552659
...,...,...,...,...,...,...,...,...,...
258,0.000168611097013,256,0.0679149669603,256,Williamsburg (South Side),Brooklyn,"MULTIPOLYGON (((-73.95834 40.71331, -73.95681 ...",-73.959905,40.710880
259,0.000394552487366,259,0.126750305191,259,Woodlawn/Wakefield,Bronx,"MULTIPOLYGON (((-73.85107 40.91037, -73.85207 ...",-73.852215,40.897932
260,0.000422345326907,260,0.133514154636,260,Woodside,Queens,"MULTIPOLYGON (((-73.90175 40.76078, -73.90147 ...",-73.906306,40.744235
261,0.0000343423231652,261,0.0271204563616,261,World Trade Center,Manhattan,"MULTIPOLYGON (((-74.01333 40.70503, -74.01327 ...",-74.013023,40.709139


In [8]:
df_place= df.loc[:,'location_id':'borough']

In [9]:
df_place.rename(columns={'location_id':'PULocationID'
   }, inplace=True)
df_place

,PULocationID,zone,borough
0,1,Newark Airport,EWR
1,2,Jamaica Bay,Queens
2,3,Allerton/Pelham Gardens,Bronx
3,4,Alphabet City,Manhattan
4,5,Arden Heights,Staten Island
...,...,...,...
258,256,Williamsburg (South Side),Brooklyn
259,259,Woodlawn/Wakefield,Bronx
260,260,Woodside,Queens
261,261,World Trade Center,Manhattan


In [10]:
df_place.to_csv('borough.cvs')

In [11]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   shape_area   263 non-null    object  
 1   objectid     263 non-null    object  
 2   shape_leng   263 non-null    object  
 3   location_id  263 non-null    object  
 4   zone         263 non-null    object  
 5   borough      263 non-null    object  
 6   geometry     263 non-null    geometry
 7   lon          263 non-null    float64 
 8   lat          263 non-null    float64 
dtypes: float64(2), geometry(1), object(6)
memory usage: 18.6+ KB


In [12]:
df['location_id']= df['location_id'].astype('int')

In [13]:
df_loc= df.loc[:,'lon':'lat']

In [14]:
df_loc['PULocationID'] = df['location_id']

In [15]:
df_loc.tail(10)

,lon,lat,PULocationID
253,-73.848641,40.832102,250
254,-74.125348,40.616880,251
255,-73.815657,40.788193,252
256,-73.858948,40.882157,254
257,-73.957418,40.718804,255
258,-73.959905,40.710880,256
259,-73.852215,40.897932,259
260,-73.906306,40.744235,260
261,-74.013023,40.709139,261
262,-73.946510,40.775932,262


In [16]:
df_loc.rename(columns={'lon':'PULocation_lon',
    'lat':'PULocation_lat'}, inplace=True)
df_loc

,PULocation_lon,PULocation_lat,PULocationID
0,-74.174000,40.691831,1
1,-73.831299,40.616745,2
2,-73.847422,40.864474,3
3,-73.976968,40.723752,4
4,-74.188484,40.552659,5
...,...,...,...
258,-73.959905,40.710880,256
259,-73.852215,40.897932,259
260,-73.906306,40.744235,260
261,-74.013023,40.709139,261


In [17]:
df_loc.to_csv('PULocation_lon_lat.cvs')

In [18]:
df_loc1= df.loc[:,'lon':'lat']

In [19]:
df_loc1

,lon,lat
0,-74.174000,40.691831
1,-73.831299,40.616745
2,-73.847422,40.864474
3,-73.976968,40.723752
4,-74.188484,40.552659
...,...,...
258,-73.959905,40.710880
259,-73.852215,40.897932
260,-73.906306,40.744235
261,-74.013023,40.709139


In [20]:
df_loc1['DOLocationID'] = df['location_id']

In [21]:
df_loc1.rename(columns={'lon':'DOLocationID_lon',
    'lat':'DOLocationID_lat'}, inplace=True)
df_loc1

,DOLocationID_lon,DOLocationID_lat,DOLocationID
0,-74.174000,40.691831,1
1,-73.831299,40.616745,2
2,-73.847422,40.864474,3
3,-73.976968,40.723752,4
4,-74.188484,40.552659,5
...,...,...,...
258,-73.959905,40.710880,256
259,-73.852215,40.897932,259
260,-73.906306,40.744235,260
261,-74.013023,40.709139,261


In [22]:
df_loc1.to_csv('DOLocationID_lon_lat.cvs')